## Clustering and Gene Identification

### Clustering

In [ ]:
library(Seurat)
library(sceasy)
library(reticulate)
library(Mfuzz)

In [ ]:
obj=readRDS('CellType.rds')  


data_path <- "mfuzz/average_expression_by_batch_and_gene.csv"
expression_data <- read.csv(data_path, row.names = 1, check.names = FALSE)
gene_expr_matrix <- as.matrix(expression_data)
eset <- new("ExpressionSet",exprs = gene_expr_matrix)

eset <- filter.NA(eset, thres = 0.25)
eset <- fill.NA(eset,mode="knn")

eset <- standardise(eset)
eset <- fill.NA(eset,mode="knn")

c <- 9 #fuzzy c-means
m <- mestimate(eset) 
set.seed(1234)
cl <- mfuzz(eset, c = c, m = m) 

library(RColorBrewer)
Color <- colorRampPalette(rev(c("#ff0000", "Yellow", "OliveDrab1")))(1000)
mfuzz.plot(eset,cl,mfrow=c(3,3),new.window= FALSE,
           time.labels=colnames(gene_expr_matrix),colo = Color)
for (i in 1:max(cl)) {
  mfuzz.plot2(eset, cl, mfrow = c(3, 3), time.labels = colnames(gene_expr_matrix),
              ylim.set = c(0, 0), xlab = "species", ylab = "",
              x11 = FALSE, ax.col = "black", bg = "white", colo = Color,
              col.axis = "black", col.lab = "black", col.main = "black", col.sub = "black", col = "black",
              centre = FALSE, centre.col = "black", centre.lwd = 2,
              cex.main = 2, cex.lab = 2, cex.axis = 1.5,
              Xwidth = 5, Xheight = 5, which.cluster = i)
}

cluster_data <- data.frame(Gene = names(cl[["cluster"]]), Cluster = as.character(cl[["cluster"]]))


write.csv(cluster_data, file = "gene_cluster_result.csv", row.names = FALSE)

### Gene Identification

#### step1 DEGs: python code

In [ ]:
adata=sc.read_h5ad('CellType.h5ad')

In [ ]:
filename = "JHvsDU"
sc.tl.rank_genes_groups(adata, groupby='BATCH', groups=['JH_JE'], reference='DUR_JE', method='wilcoxon', corr_method='benjamini-hochberg', key_added=filename)
result = adata.uns[filename]
groups = result['names'].dtype.names
marker=pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', 'scores', 'pvals_adj', 'logfoldchanges']})
marker.columns = ['gene', 'scores', 'Padj', 'log2FC']
marker.to_csv(filename+'.csv')

In [ ]:
filename = "JHvsAWB"
sc.tl.rank_genes_groups(adata, groupby='BATCH', groups=['JH_JE'], reference='SS_JE', method='wilcoxon', corr_method='benjamini-hochberg', key_added=filename)
result = adata.uns[filename]
groups = result['names'].dtype.names
marker=pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', 'scores', 'pvals_adj', 'logfoldchanges']})
marker.columns = ['gene', 'scores', 'Padj', 'log2FC']
marker.to_csv(filename+'.csv')

In [ ]:
adata.obs['CellType'].unique() 

In [ ]:
newdata = adata[adata.obs['CellType'].isin(['Plasma'])]

In [ ]:
filename = "JHvsDU_Plasma"
sc.tl.rank_genes_groups(newdata, groupby='BATCH', groups=['JH_JE'], reference='DUR_JE', method='wilcoxon', corr_method='benjamini-hochberg', key_added=filename)
result = newdata.uns[filename]
groups = result['names'].dtype.names
marker=pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', 'scores', 'pvals_adj', 'logfoldchanges']})
marker.columns = ['gene', 'scores', 'Padj', 'log2FC']
marker.to_csv(filename+'.csv')

In [ ]:
filename = "JHvsAWB_Plasma"
sc.tl.rank_genes_groups(newdata, groupby='BATCH', groups=['JH_JE'], reference='SS_JE', method='wilcoxon', corr_method='benjamini-hochberg', key_added=filename)
result = newdata.uns[filename]
groups = result['names'].dtype.names
marker=pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', 'scores', 'pvals_adj', 'logfoldchanges']})
marker.columns = ['gene', 'scores', 'Padj', 'log2FC']
marker.to_csv(filename+'.csv')

#### step2 Intersect gene: R code

In [ ]:
library(ggplot2)
library(ggforce)
library(dplyr)
library(plyr)

In [ ]:
data1 <- read.csv("/disk213/xieqq/Fuwy_re/je/immune/diffirent gene/JHvsAWB_CD4 αβ T cells.csv",row.names=1)
data1 <- data1[order(-data1$log2FC), ]
data1$Type <- "none"
data1[which(data1$Padj<0.05 & data1$log2FC>1.5),"Type"] <- "up"
data1[which(data1$Padj<0.05 & data1$log2FC<(-1.5)),"Type"] <- "down"
data1$group <- "JHvsDU"

data2 <- read.csv("JHvsDU_CD4 αβ T cells.csv",row.names=1)
data2 <- data2[order(-data2$log2FC), ]
data2$Type <- "none"
data2[which(data2$Padj<0.05 & data2$log2FC>1.5),"Type"] <- "down"
data2[which(data2$Padj<0.05 & data2$log2FC<(-1.5)),"Type"] <- "up"
data2$group <- "JHvsAWB"

In [ ]:
type1 <- data.frame(gene=intersect(data1[which(data1$Type=="up"),]$gene,data2[which(data2$Type=="up"),]$gene),Type="type1")
type2 <- data.frame(gene=intersect(data1[which(data1$Type=="up"),]$gene,data2[which(data2$Type=="down"),]$gene),Type="type2")
type3 <- data.frame(gene=intersect(data1[which(data1$Type=="down"),]$gene,data2[which(data2$Type=="up"),]$gene),Type="type3")
type4 <- data.frame(gene=intersect(data1[which(data1$Type=="down"),]$gene,data2[which(data2$Type=="down"),]$gene),Type="type4")
out <- rbind(rbind(type1,type2),rbind(type3,type4))

In [ ]:
cluster <- read.csv("gene_cluster_result.csv")
out_immune <- left_join(out,cluster,by=c("gene"="Gene"))
write.csv(out_immune,"Intersect_Immune.csv",row.names=F)

cluster <- read.csv("plasma_gene_cluster_result.csv")
out_plasma <- left_join(out,cluster,by=c("gene"="Gene"))
write.csv(out_plasma,"Intersect_Plasma.csv",row.names=F)

cluster <- read.csv("CD4_gene_cluster_result.csv")
out_CD4 <- left_join(out,cluster,by=c("gene"="Gene"))
write.csv(out_CD4,"Intersect_CD4.csv",row.names=F)

cluster <- read.csv("CD8_gene_cluster_result.csv")
out_CD8 <- left_join(out,cluster,by=c("gene"="Gene"))
write.csv(out_CD8,"Intersect_CD8.csv",row.names=F)

In [ ]:
out <- read.csv("Intersect_Immune.csv",check.names=F)

In [ ]:
result <- out %>% group_by(Type) %>% summarise(count=n()) 
ppie <- ggplot()+
  geom_arc_bar(data=result,stat="pie",aes(x0=0,y0=0,r0=1,r=2,amount=count,fill=Type))+
  xlab("")+ylab('')+
  scale_fill_manual(values=c('type1'='#02CECB','type2'='#FCF300','type3'='#1E96FC','type4'='#E12C2C'))+
  theme_bw()+
  theme(axis.ticks=element_blank(), 
        axis.text.y=element_blank(),
        axis.text.x=element_blank(),
        legend.position="none",
        panel.border=element_blank(),
        panel.background=element_blank(),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())
ppie
ggsave(filename="./figures/Pie_CD8.pdf", plot=ppie, width=4, height=4)

In [ ]:
result <- out %>% group_by(Type,Cluster) %>% summarise(count=n(), .groups="drop")
result <- ddply(result, "Type", transform, percent_weight=count/sum(count)*100)
pbar <- ggplot(result, aes(x=Type, y=percent_weight, fill=Cluster)) +
  geom_bar(stat="identity",width=0.5)+
  scale_fill_gradient(low = "#FCF300", high = "#1E96FC")+
  xlab("")+ylab('')+
  theme_bw()+
  theme(axis.text.x=element_text(color="black", size=10),
        axis.text.y=element_text(color="black", size=10),
        axis.title.x=element_text(color="black",size=12),
        axis.title.y=element_text(color="black",size=12),
        legend.text=element_text(color="black",size=10),
        legend.title=element_text(color="black",size=12),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())
pbar
ggsave(filename="./figures/Bar_CD8.pdf", plot=pbar, width=4, height=4)

#### step3 dotplot: python code

In [ ]:
os.chdir("Intersect")

In [ ]:
gene_dict={'type1':["ATP8","ISG12(A)","ENSSSCG00000024914","SAMD12","ATP2C2"], #CFB
           'type2':["CD40LG","PIGR","TNFRSF17","CCR10","SYK"],
           'type3':["CCL5","XCL1","PTPN18","PTPN3","PPP1R15A","CEBPB"],
           'type4':["SLC7A11"]
}

newdata = adata
sc.pl.dotplot(newdata, var_names=gene_dict, groupby=['BREED'],vmin=0,vmax=2,dot_min=0,dot_max=1,save='Intersect_Immune.pdf')

In [ ]:
gene_dict={'type1':["ATP8"],
           'type2':["JCHAIN","ENSSSCG00000037775","CST3","HSP90B1","PDIA4"],  #IGHM
           'type3':['CCL5','GZMH','SKAP1','HSPH1','GNLY']
}

newdata = adata[adata.obs['CellType'].isin(['Plasma'])]
sc.pl.dotplot(newdata, var_names=gene_dict, groupby=['BREED'],vmin=0,vmax=2,dot_min=0,dot_max=1,save='Intersect_Plasma.pdf')

In [ ]:
gene_dict={'type2':["HBB","IL23R","MZB1","ENSSSCG00000001521","RASGRP3"],
           'type3':["CCL5","CD81",'AZIN1',"XCL1",'NR4A3']
}

newdata = adata[adata.obs['CellType'].isin(['CD4 αβ T cells'])]
sc.pl.dotplot(newdata, var_names=gene_dict, groupby=['BREED'],vmin=0,vmax=2,dot_min=0,dot_max=1,save='Intersect_CD4.pdf')

In [ ]:
gene_dict={'type1':["ENSSSCG00000038182","ENSSSCG00000024914","SAMD12"],
           'type2':["HBB","ENSSSCG00000001521","C1QC","DNASE1L3","MMP14"],
           'type3':['GNLY','CLCA1','FABP6','TMEM238','RND1']
}

newdata = adata[adata.obs['CellType'].isin(['CD8 αβ T cells'])]
sc.pl.dotplot(newdata, var_names=gene_dict, groupby=['BREED'],vmin=0,vmax=2,dot_min=0,dot_max=1,save='Intersect_CD8.pdf')

In [ ]:
data = pd.read_csv("Intersect_Immune.csv")
data = pd.read_csv("Intersect_Plasma.csv")
data = pd.read_csv("Intersect_CD4.csv")
data = pd.read_csv("Intersect_CD8.csv")

In [ ]:
for i in data['Type'].unique()[:]:
    gene_list = data[data['Type'].isin([i])]
    enr = gp.enrichr(gene_list=gene_list['gene'],
                     gene_sets='GO_Biological_Process_2023',  #['GO_Biological_Process_2023','GO_Cellular_Component_2023','GO_Molecular_Function_2023']
                     organism='Human',
                     outdir='./',
                     cutoff=0.05, 
                     no_plot=True
                    )
    enr.results = enr.results[enr.results['Adjusted P-value'] < 0.05]
    enr.results['Group'] = i
    enr.results['-Log10 P-value'] = -enr.results['Adjusted P-value'].apply(math.log10)
    enr.results['Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[0])
    enr.results['Gene Count'] = gene_list.shape[0]
    enr.results['Background Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[1])
    enr.results['Count'] = pd.to_numeric(enr.results['Count'],errors='coerce')
    enr.results['Gene Count'] = pd.to_numeric(enr.results['Gene Count'],errors='coerce')
    enr.results['Background Count'] = pd.to_numeric(enr.results['Background Count'],errors='coerce')
    enr.results['Fold Enrichment'] = (enr.results['Count']/enr.results['Gene Count'])/(enr.results['Background Count']/14937)
    
    enr.results=enr.results.sort_values(by='Fold Enrichment', ascending=False)
    enr.results=enr.results.reset_index(drop = True)
    enr.results.to_csv('CD8_'+i+'_GO.csv')

In [ ]:
for i in data['Type'].unique()[:]:
    gene_list = data[data['Type'].isin([i])]
    enr = gp.enrichr(gene_list=gene_list['gene'],
                     gene_sets='KEGG_2021_Human',
                     organism='Human',
                     outdir='./',
                     cutoff=0.05, 
                     no_plot=True
                    )
    enr.results = enr.results[enr.results['Adjusted P-value'] < 0.05]
    enr.results['Group'] = i
    enr.results['-Log10 P-value'] = -enr.results['Adjusted P-value'].apply(math.log10)
    enr.results['Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[0])
    enr.results['Gene Count'] = gene_list.shape[0]
    enr.results['Background Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[1])
    enr.results['Count'] = pd.to_numeric(enr.results['Count'],errors='coerce')
    enr.results['Gene Count'] = pd.to_numeric(enr.results['Gene Count'],errors='coerce')
    enr.results['Background Count'] = pd.to_numeric(enr.results['Background Count'],errors='coerce')
    enr.results['Fold Enrichment'] = (enr.results['Count']/enr.results['Gene Count'])/(enr.results['Background Count']/8078)
    
    enr.results=enr.results.sort_values(by='Fold Enrichment', ascending=False)
    enr.results=enr.results.reset_index(drop = True)
    enr.results.to_csv('CD8_'+i+'_KEGG.csv')